<h1>Preparation of the German Twitter Data</h1>

<p>This file is for preparing the german tweets. The preparation is mainly for training my own model for sentiment detection with Flair. </p>

In [1]:
import csv
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from HanTa import HanoverTagger as ht

<p>Download der Stopwörter Listen</p>

In [ ]:
#nltk.download('stopwords')

<h1>Einlesen und bestimmung der Felder</h1>
<p>Durch die weitern Operatoren, kann ich die einzelnen Felder in DF bestimmen. Durch die Lambda Funktion werden die Str zu Listen, was die spätere verarbeitung erleichtert. 
    Durch die Parse_dates Funktion wird das Datumsfeld zu einem Timestamp, was die weiter Verarbeitung wesentlich leichter macht.</p>

In [2]:
df_manager_de = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Manager_DE/data.csv", sep=";", 
                            converters={"tweet_type": lambda x: x.strip("[]").replace("'", "").split(", "),
                                        "referenced_tweet_id": lambda x: x.strip("[]").replace("'", "").split(", ")},
                           parse_dates=["created_at"])
df_manager_de = df_manager_de.explode(["referenced_tweet_id", "tweet_type"], ignore_index=True)

In [3]:
df_manager_de.head()

conversation_id   author_id                created_at  \
0  703311366262800384  1392131804 2016-02-26 20:11:11+00:00   
1  702405572012974080  1053723204 2016-02-24 08:11:53+00:00   
2  701784983829143553   554491983 2016-02-22 15:05:53+00:00   
3  701446667170615296  3365669884 2016-02-21 16:41:32+00:00   
4  700597812556386304  4858383167 2016-02-19 08:28:29+00:00   

             tweet_id lang  retweet_count  reply_count  like_count  \
0  703311366262800384   de              0            0           2   
1  702405572012974080   de              0            0           0   
2  701784983829143553   de              0            0           0   
3  701446667170615296   de              0            0           1   
4  700597812556386304   de              2            0           0   

   quote_count              source tweet_type referenced_tweet_id  \
0            0  Twitter Web Client       None                None   
1            0  Twitter Web Client       None                None   
2            0            Facebook       None                None   
3            0  Twitter Web Client       None                None   
4            0  Twitter for iPhone  retweeted  700578193334411264   

                                                text  
0  #Wirecard-Chef Markus Braun pumpt irrwitzige S...  
1  Jetzt günstig einsammeln und später absahnen? ...  
2  Morgen startet die neue Reihe R²-JazzCube! Vol...  
3  die Y-Achse des  #prusai3 nimmt Formen an :-) ...  
4  RT @aktionaer: #Wirecard: Was führt der Vorsta...

<br>
<p>Prüfen welche Datenfelder in den einzelnen Zeilen enthalten sind. War wichtig um diese anpassen zu können.</p>

In [ ]:
print("conversation_id: ", df_manager_de.iloc[0]["conversation_id"], "type: ", type(df_manager_de.iloc[0]["conversation_id"]))
print("author_id: ", df_manager_de.iloc[0]["author_id"], "type: ", type(df_manager_de.iloc[0]["author_id"]))
print("created_at: ", df_manager_de.iloc[0]["created_at"], "type: ", type(df_manager_de.iloc[0]["created_at"]))
print("tweet_id: ", df_manager_de.iloc[0]["tweet_id"], "type: ", type(df_manager_de.iloc[0]["tweet_id"]))
print("lang: ", df_manager_de.iloc[0]["lang"], "type: ", type(df_manager_de.iloc[0]["lang"]))
print("retweet_count: ", df_manager_de.iloc[0]["retweet_count"], "type: ", type(df_manager_de.iloc[0]["retweet_count"]))
print("reply_count: ", df_manager_de.iloc[0]["reply_count"], "type: ", type(df_manager_de.iloc[0]["reply_count"]))
print("like_count: ", df_manager_de.iloc[0]["like_count"], "type: ", type(df_manager_de.iloc[0]["like_count"]))
print("quote_count: ", df_manager_de.iloc[0]["quote_count"], "type: ", type(df_manager_de.iloc[0]["quote_count"]))
print("source: ", df_manager_de.iloc[0]["source"], "type: ", type(df_manager_de.iloc[0]["source"]))
print("tweet_type: ", df_manager_de.iloc[0]["tweet_type"], "type: ", type(df_manager_de.iloc[0]["tweet_type"]))
print("referenced_tweet_id: ", df_manager_de.iloc[0]["referenced_tweet_id"], "type: ", type(df_manager_de.iloc[0]["referenced_tweet_id"]))
print("text: ", df_manager_de.iloc[0]["text"], "type: ", type(df_manager_de.iloc[0]["text"]))

<h3>Löschen von unnötigen Informationen</h3>
<p>Hier werden Informationen aus den Tweets Texten, die nicht weiter benötigt werden. Bspw. Links</p>

In [4]:
for each in range(len(df_manager_de)):
    df_manager_de.loc[df_manager_de["text"] == 
                      df_manager_de.loc[each]["text"], "text"] = re.sub(r'http\S+', '', 
                                                                        (re.sub('@[^\s]+', '',
                                                                                re.sub('[#|$]', '', 
                                                                                       df_manager_de.loc[each]["text"]))))

<p>Dieser Teil wurde für das Testen gebraucht.</p>

In [ ]:
#just for test
for each in range(len(df_manager_de)):
    df_manager_de.loc[df_manager_de["text"] == 
                      df_manager_de.loc[each]["text"], "text"] = re.sub(r'http\S+', '',(re.sub('[#|$]', '', df_manager_de.loc[each]["text"])))

In [5]:
df_manager_de.iloc[0]["text"]

'Wirecard-Chef Markus Braun pumpt irrwitzige Summen in sein Unternehmen. Woher stammt das Geld?   '

In [6]:
len(df_manager_de)

14451

<h3>Nicht Kontextbezogene Tweets löschen</h3>
<p>So kann ich prüfen ob bestimmte Begriffe in den Texten enthalten sind und mir die conversation_id ausgeben lassen. Diese lasse ich dann löschen und auch zugehörige Tweets, da diese nicht in den Kontext passen.<br>Die wörter in der Liste müssen klein geschrieben werden, da auch alle Buchstaben für die Prüfung klein geschrieben werden müssen, da sinst nicht alles erkannt wird.</p>
<p>Zu löschende Begriffe</p>
<ul>
    <li>BVB</li>
    <li>Fußball</li>
    <li>Fussball</li>
    <li>Dortmund</li>
    <li>kicker</li>
    <li>fpö</li>
    <li>fpoe</li>
</ul>

In [7]:
not_used_words_list = ["bvb", "fußball", "fussball", "dortmund", "fpö", "fpoe", "kicker"]
tweet_id_without_context_list = []
for each in range(len(df_manager_de)):
    for word in not_used_words_list:
        if word in df_manager_de.iloc[each]["text"].lower():
            if df_manager_de.iloc[each]["tweet_id"] not in tweet_id_without_context_list:
                tweet_id_without_context_list.append(df_manager_de.iloc[each]["tweet_id"])

<p>Iteration durch die Liste mit zu löschenden Tweets, bis der Wert zu Beginn und zum Ende gleich ist und es keine
weiteren Tweets mehr gibt, die gefiltert werden müssen. </p>

In [8]:
def check_for_further_tweets():
    start_len = len(tweet_id_without_context_list)
    for each in range(len(tweet_id_without_context_list)):
        df_tweets = df_manager_de.loc[(df_manager_de["conversation_id"] == tweet_id_without_context_list[each]) | 
               (df_manager_de["referenced_tweet_id"] == str(tweet_id_without_context_list[each]))]

        for each in range(len(df_tweets)):
            if df_tweets.iloc[each]["tweet_id"] not in tweet_id_without_context_list:
                tweet_id_without_context_list.append(df_tweets.iloc[each]["tweet_id"])
                
    if start_len != len(tweet_id_without_context_list):
        check_for_further_tweets()
check_for_further_tweets()

<p>Löschen der nicht kontextbezogenen Tweets und den Index neu setzen um später keine Probleme mit dem Index zu bekommen.</p>

In [9]:
for each in tweet_id_without_context_list:
    df_manager_de = df_manager_de.drop(df_manager_de.loc[df_manager_de["tweet_id"] == each].index[0])
df_manager_de = df_manager_de.reset_index(drop=True)

<p>Löschen und Ermittlung von Tweet_IDs, die keinen Kontext zum Thema haben.</p>

In [10]:
#Text Check up
list_tweet_id = [701784983829143553, 909831998537158657, 909733713302441984, 909730731932368897, 1115628103487954944, 
                1115596390133051393, 1112624842493358080, 1131274228521742336, 1130213579490234370, 1207617572126044160, 
                1265176607905333248, 1273949458808680449, 1285646389771481088, 1285547240447246338, 1285515450584096768, 
                1285497151125151745, 1284925162836709378, 1279724802782216192]
list_tweet_id_remove = []
for tweet in list_tweet_id:
    for each in df_manager_de.loc[(
        df_manager_de["tweet_id"] == tweet) | (
        df_manager_de["conversation_id"] == df_manager_de.loc[df_manager_de["tweet_id"] == tweet]["conversation_id"].values[0]) | (
        df_manager_de["referenced_tweet_id"] == tweet) | (
        (df_manager_de["referenced_tweet_id"] == df_manager_de.loc[df_manager_de["tweet_id"] == tweet]["referenced_tweet_id"].values[0]) & (df_manager_de["referenced_tweet_id"] != "None")) | (
        df_manager_de["referenced_tweet_id"] == df_manager_de.loc[df_manager_de["tweet_id"] == tweet]["conversation_id"].values[0])]["tweet_id"].values:
        #print(each, "\n")
        if each not in list_tweet_id_remove:
            list_tweet_id_remove.append(each)

<p>Löschen der zuvor ermitteletn Tweets.</p>

In [11]:
for each in list_tweet_id_remove:
    df_manager_de = df_manager_de.drop(df_manager_de.loc[df_manager_de["tweet_id"] == each].index[0])
df_manager_de = df_manager_de.reset_index(drop=True)

<br>
<h3>Löschen von Retweets</h3>
<p>Alle Retweets löschen, da diese in diesem Schritt nicht benötigt werden.</p>

In [12]:
counter = 0
retweet_list = []
for each in range(len(df_manager_de)):
    if "retweeted" in df_manager_de.iloc[each]["tweet_type"]:
        retweet_list.append(each)
        counter += 1
print(counter)

5280


In [13]:
df_manager_de = df_manager_de.drop(retweet_list)
df_manager_de = df_manager_de.reset_index(drop=True)

<h3>Löschen von doppelten Tweets</h3>
<p>Alle Tweets, die dopplet im Datensatz enthalten sind, anhand der tweet_id werden gelöscht. Somit werden auch die zuvor aufgespalteten Listen wieder gelöscht</p>

In [14]:
df_manager_de = df_manager_de.drop_duplicates(subset=["tweet_id"])
df_manager_de = df_manager_de.reset_index(drop=True)

In [15]:
len(df_manager_de)

8707

<h1>Check User</h1>
<p>Überprüfe die User Daten nach Namen, die nichts mit dem Thema Wirecard zu tun haben. Auch Nutzer, die ggf. nur durch Zufall in die Abfrage gekommen sind, z.B. durch den gleichen Namen wie einer der Manager, sollen aus dem Datensatz entnommen werden.<br>Auffällige Namen sind <b>Markus Braun</b> und <b>Jan Marsalek</b>, da es sich um den CEO und den verantwortlichen Manager handelt.</p>

<p>Laden der Daten</p>

In [16]:
df_user = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Manager_DE/user.csv", sep=";")

<h3>Nutzer die keinen Mehrwert bieten werden gelöscht.</h3>

In [17]:
list_not_needed_author_ids = [3365669884, 536335486]
for each in list_not_needed_author_ids:
    df_manager_de = df_manager_de.drop(df_manager_de.loc[df_manager_de["author_id"] == each].index)
df_manager_de = df_manager_de.reset_index(drop=True)

<h2>Überprüfung und Check der Twitter Nutzer im Datensatz</h2>
<p>Dies ist für den Fall relevant, dass die Nutzer überprüft werden müssen. <br>Im aktuellen Zustand des Datensatzes nicht mehr notwendig.</p>

<p>Prüfen nach allen Nutzern mit dem Namen Markus</p>

In [ ]:
df_user.loc[df_user.name.str.lower().str.contains("markus")].drop_duplicates()

<p>Prüfen auf auffäliige Einträge bezüglich Jan Marksalek</p>

In [ ]:
df_user.loc[df_user.name.str.lower().str.contains("jan")].drop_duplicates()

<p>Auffällige User_IDs:</p>
<ul>Markus Braun
    <li>3365669884 - MarkusB1980</li>
    <li>827452635057881088 - _MarkusBraun( - CWO von Wirecard)</li>
    <li>536335486 - Advobraun</li>
    <li>780754694419296256 - markuscbraun1</li>
</ul>
<ul>Jan (Marsalek)
    <li>19072286 - janboehm</li>
</ul>

<p>Anzeigen der Tweets der "auffälligen" Nutzer</p>

In [ ]:
#3365669884 - MarkusB1980
pd.set_option('display.max_colwidth', None)
df_manager_de.loc[df_manager_de["author_id"] == 3365669884]["text"]

In [ ]:
#827452635057881088 - _MarkusBraun( - CWO von Wirecard)
pd.set_option('display.max_colwidth', None)
df_manager_de.loc[df_manager_de["author_id"] == 827452635057881088]["text"]

In [ ]:
#536335486 - Advobraun
pd.set_option('display.max_colwidth', None)
df_manager_de.loc[df_manager_de["author_id"] == 536335486]["text"]

In [ ]:
#780754694419296256 - markuscbraun1
pd.set_option('display.max_colwidth', None)
df_manager_de.loc[df_manager_de["author_id"] == 780754694419296256]["text"]

In [ ]:
#19072286 - janboehm
pd.set_option('display.max_colwidth', None)
df_manager_de.loc[df_manager_de["author_id"] == 19072286]["text"]

<p>Nur bei zwei der Nutzer ist nach der Vorverarbeitung noch ein Tweet in der Liste. Diese beiden Nutzer tragen nichts zum Thema Wirecard bei und twittern nicht diesbezüglich. <br>Ich habe Jan bei der Suche nach Jan lediglich den Namen Jan Böhmermann geprüft, da es sich hierbei um eine bekannte Persönlichkeit handelt, die zu diesem aber auch zu vielen anderen Themen twittert und ich dies deswegen überprüfen wollte.<br><br>Prüfen auf die Nutzer, die besonders oft im Datensatz der Manager vorkommen.<br>Nutzer, deren Inhalt keinen Mehrwert bieten: </p>
<lu>
    <li>3365669884 - MarkusB1980</li>
    <li>536335486 - Advobraun</li>
</lu>
<p>Dabei handelt es sich um die einzigen Nutzer, zu denen noch Tweets nach der Vorverarbeitung übrig sind</p>

<p>Das Dataframe mit der Anzahl der jeweiligen Autoren wird um die Informationen aus dem Datensatz der User ergänzt.<br>Dabei soll gerpüft werden, ob auffällige Nutzer mit eigenartigen Namen auftauchen, die es zu prüfen gilt. Dabei wird vor allem auf Nutzer geachtet, die einen klaren Bezug zu anderen Theman haben oder die ggf. auffälligkeiten zu dem Thema haben könnten.</p>

In [ ]:
df_manager_de["author_id"].value_counts().to_frame().reset_index().join(
    df_user.set_index("user_id"), on="index").drop_duplicates(subset=["index"]).rename(columns={"index": "author_id", "author_id": "count"}).head(11)

<p>Stichprobenartige Prüfen der einzelnen Top Nutzer mit mehr als 50 Tweets auf deren Inhalte.</p>

In [ ]:
pd.set_option('display.max_colwidth', None)
for each in df_manager_de.loc[df_manager_de["author_id"] == 1118219555967520769].index:
    print(df_manager_de.iloc[each]["text"], "\n")

In [ ]:
for each in df_manager_de.loc[df_manager_de["author_id"] == 1159698800790953984].index:
    print(df_manager_de.iloc[each]["text"], "\n")

In [ ]:
for each in df_manager_de.loc[df_manager_de["author_id"] == 3029833842].index:
    print(df_manager_de.iloc[each]["text"], "\n")

In [ ]:
for each in df_manager_de.loc[df_manager_de["author_id"] == 2195833356].index:
    print(df_manager_de.iloc[each]["text"], "\n")

In [ ]:
for each in df_manager_de.loc[df_manager_de["author_id"] == 704753138146877440].index:
    print(df_manager_de.iloc[each]["text"], "\n")

<h1>Hinzufügen weiterer Twitter Datensätze</h1>

<h2>Datensatz mit Abfrage nach Wirecard</h2>

In [18]:
df_wirecard_de = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Wirecard_DE/data.csv", sep=";", 
                            converters={"tweet_type": lambda x: x.strip("[]").replace("'", "").split(", "),
                                        "referenced_tweet_id": lambda x: x.strip("[]").replace("'", "").split(", ")},
                           parse_dates=["created_at"])
df_wirecard_de = df_wirecard_de.explode(["referenced_tweet_id", "tweet_type"], ignore_index=True)


In [19]:
for each in range(len(df_wirecard_de)):
    df_wirecard_de.loc[df_wirecard_de["text"] == 
                      df_wirecard_de.loc[each]["text"], "text"] = re.sub(r'http\S+', '', 
                                                                        (re.sub('@[^\s]+', '',
                                                                                re.sub('[#|$]', '', 
                                                                                       df_wirecard_de.loc[each]["text"]))))

In [ ]:
df_wirecard_de.head()

<h3>Löschen von Retweets</h3>

In [20]:
counter = 0
retweet_list = []
for each in range(len(df_wirecard_de)):
    if "retweeted" in df_wirecard_de.iloc[each]["tweet_type"]:
        retweet_list.append(each)
        counter += 1
print(counter)
df_wirecard_de = df_wirecard_de.drop(retweet_list)
df_wirecard_de = df_wirecard_de.reset_index(drop=True)

54403


In [21]:
len(df_wirecard_de)

111531

<h3>Löschen von doppelten Einträgen</h3>

In [22]:
df_wirecard_de = df_wirecard_de.drop_duplicates(subset=["tweet_id"])
df_wirecard_de = df_wirecard_de.reset_index(drop=True)

In [23]:
len(df_wirecard_de)

111254

In [24]:
df_wirecard_de.head()

conversation_id   author_id                created_at  \
0  704018583765000194  3000333467 2016-02-28 19:01:25+00:00   
1  704016062682103808  3383023023 2016-02-28 18:51:23+00:00   
2  703952749650714624    20973609 2016-02-28 14:39:48+00:00   
3  703097227355430912  3296434535 2016-02-28 10:37:51+00:00   
4  703887151348457472   163466981 2016-02-28 10:19:09+00:00   

             tweet_id lang  retweet_count  reply_count  like_count  \
0  704018583765000194   de              0            0           0   
1  704016062682103808   de              0            0           2   
2  703952749650714624   de              0            0           0   
3  703891857542127616   de              0            0           0   
4  703887151348457472   de              0            0           0   

   quote_count              source  tweet_type referenced_tweet_id  \
0            0  Twitter for iPhone        None                None   
1            0  Twitter Web Client        None                None   
2            0  Twitter for iPhone        None                None   
3            0  Twitter Web Client  replied_to  703101750123720704   
4            0              Aktien        None                None   

                                                text  
0  3 lukrative Aktien im Quick-Check.True Gold/Wi...  
1  Die Woche mit Fintech und Banken im Überblick ...  
2  Wirecard / Deutsche Telekom / DAX / Euro / Gol...  
3     weil wir letztens darüber geschrieben haben...  
4  Aktien, Banken, Makrodaten, S&amp;P 500 - Bärm...

<h2>Abfrage nach Chashtag Wirecard</h2>

In [25]:
df_wrcdf_de = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/WRCDF_DE/data.csv", sep=";", 
                            converters={"tweet_type": lambda x: x.strip("[]").replace("'", "").split(", "),
                                        "referenced_tweet_id": lambda x: x.strip("[]").replace("'", "").split(", ")},
                           parse_dates=["created_at"])
df_wrcdf_de = df_wrcdf_de.explode(["referenced_tweet_id", "tweet_type"], ignore_index=True)


In [26]:
for each in range(len(df_wrcdf_de)):
    df_wrcdf_de.loc[df_wrcdf_de["text"] == 
                      df_wrcdf_de.loc[each]["text"], "text"] = re.sub(r'http\S+', '', 
                                                                        (re.sub('@[^\s]+', '',
                                                                                re.sub('[#|$]', '', 
                                                                                       df_wrcdf_de.loc[each]["text"]))))

In [27]:
len(df_wrcdf_de)

33

<h3>Löschen von Retweets</h3>

In [28]:
counter = 0
retweet_list = []
for each in range(len(df_wrcdf_de)):
    if "retweeted" in df_wrcdf_de.iloc[each]["tweet_type"]:
        retweet_list.append(each)
        counter += 1
print(counter)
df_wrcdf_de = df_wrcdf_de.drop(retweet_list)
df_wrcdf_de = df_wrcdf_de.reset_index(drop=True)

0


<h3>Löschen von doppelten Tweets</h3>

In [29]:
df_wrcdf_de = df_wrcdf_de.drop_duplicates(subset=["tweet_id"])
df_wrcdf_de = df_wrcdf_de.reset_index(drop=True)

In [30]:
len(df_wrcdf_de)

33

<h1>Zusammenführen der Datensätze</h1>

In [31]:
list_data_frames = [df_manager_de, df_wirecard_de, df_wrcdf_de]
df_de = pd.concat(list_data_frames, ignore_index=True)

In [32]:
print(len(df_manager_de))
print(len(df_wirecard_de))
print(len(df_wrcdf_de))
print(len(df_manager_de)+len(df_wirecard_de)+len(df_wrcdf_de))
print(len(df_de))

8703
111254
33
119990
119990


<h3>Löschen der doppelten Posts</h3>
<p>Dadurch, dass die Abfrage durch mehrere Querys erfolgt ist, befinden sich einige Tweets doppelt in dem Datensatz. Diese werden hier anhand der <b>"tweet_id"</b> gelöscht. </p>

In [33]:
df_de = df_de.drop_duplicates(subset=["tweet_id"])
df_de = df_de.reset_index(drop=True)

<p>Der neue Datensatz wird auf plausibilität geprüft.</p>

In [34]:
print(len(df_de))

114444


In [35]:
df_de.head()

conversation_id   author_id                created_at  \
0  703311366262800384  1392131804 2016-02-26 20:11:11+00:00   
1  702405572012974080  1053723204 2016-02-24 08:11:53+00:00   
2  700578193334411264    19811837 2016-02-19 07:10:32+00:00   
3  700221272840601601    19811837 2016-02-18 07:32:15+00:00   
4  697324084221227008   183289696 2016-02-10 07:39:51+00:00   

             tweet_id lang  retweet_count  reply_count  like_count  \
0  703311366262800384   de              0            0           2   
1  702405572012974080   de              0            0           0   
2  700578193334411264   de              2            0           0   
3  700221272840601601   de              0            0           1   
4  697324084221227008   de              0            0           0   

   quote_count              source tweet_type referenced_tweet_id  \
0            0  Twitter Web Client       None                None   
1            0  Twitter Web Client       None                None   
2            0  Twitter Web Client       None                None   
3            0  Twitter Web Client       None                None   
4            0             dlvr.it       None                None   

                                                text  
0  Wirecard-Chef Markus Braun pumpt irrwitzige Su...  
1  Jetzt günstig einsammeln und später absahnen? ...  
2  Wirecard: Was führt der Vorstand im Schilde? M...  
3  Wirecard-Vorstand kauft weiter massiv eigene A...  
4  📈 Wirecard: Serie von Aktienkäufen des Vorstan...

In [36]:
df_de.tail()

conversation_id            author_id                created_at  \
114439  1164960547693105153           4584354895 2019-08-23 18:00:13+00:00   
114440  1228994498610503680   704753138146877440 2020-02-17 17:53:29+00:00   
114441  1228017316375539712  1138485557502382082 2020-02-13 18:09:16+00:00   
114442  1275070899822059523           1001192245 2020-06-22 14:19:26+00:00   
114443  1271435871364042754            392388185 2020-06-12 13:45:22+00:00   

                   tweet_id lang  retweet_count  reply_count  like_count  \
114439  1164960547693105153   de              0            0           0   
114440  1229463896425205768   de              0            0           2   
114441  1228018315785838593   de              0            0           0   
114442  1275070899822059523   de              0            0           0   
114443  1271438445500682244   de              0            1           9   

        quote_count              source  tweet_type  referenced_tweet_id  \
114439            0         tickwatcher        None                 None   
114440            0     Twitter Web App  replied_to  1229459682403725314   
114441            1  Twitter for iPhone  replied_to  1228017316375539712   
114442            0      TalkMarketsApp        None                 None   
114443            0     Twitter Web App  replied_to  1271435871364042754   

                                                     text  
114439  Top Gainers (&gt;30)UVXY JNUG DRIP OPYGY URR E...  
114440                   1/ Die Anwälte der Klägerseit...  
114441  Ist nicht der WRCDF LV Anteil mit zusätzlichen...  
114442  Olaf Scholz - The German Finance Minister Is D...  
114443   12.06. WDI              29,09 Mio.  +1,16% (v...

In [37]:
df_de.iloc[2]

conversation_id                                       700578193334411264
author_id                                                       19811837
created_at                                     2016-02-19 07:10:32+00:00
tweet_id                                              700578193334411264
lang                                                                  de
retweet_count                                                          2
reply_count                                                            0
like_count                                                             0
quote_count                                                            0
source                                                Twitter Web Client
tweet_type                                                          None
referenced_tweet_id                                                 None
text                   Wirecard: Was führt der Vorstand im Schilde? M...
Name: 2, dtype: object

<p><b>Fertig</b> mit der Vorverarbeitung der Daten in deutsch. Es ist nun ein Datensatz verfügbar mit allen Tweets für den deutschsprachigen Raum. Es sind keine doppelten Tweets enthalten, solange diese keine gleiche tweet_id enthalten. Auch Nutzer und tweets, die nichts zu dem eigentlichen Thema Wirecard beitragen sind nicht mehr im Datensatz enthalten. Vereinzelt können natürlich unnütze Tweets vorhanden sein, aber die Nutzer mit vielen Interaktionen sind überprüft und auch die Begriffe, die besonders oft vorkommen sind überprüft worden, ob diese etwas mit dem Thema zu tun haben.<br>Jetzt muss das gleich noch mit den Englischen tweets geschehen und dann müssen Datensätze zur Überprüfung ausgegeben werden.<br> Um den zufälligen Datensatz zu erstellen sollte ich noch einen zufallszahlengenerator nutzen, sodass ich möglichst zufällige Tweets zum testen erhalte. Hier scheint die Pandas Funktion <b>"sample"</b> gut geeignet zu sein. </p>

In [40]:
df_de.sample(n = 20)

conversation_id            author_id                created_at  \
69503   1277525797243412480  1245423304304926727 2020-06-29 08:54:20+00:00   
78607   1275397576221765633            331867773 2020-06-23 21:29:21+00:00   
7263    1285235188633677824  1118219555967520769 2020-07-20 19:54:11+00:00   
32865   1071003895101566977           2280206959 2018-12-07 11:30:01+00:00   
62348   1255054199756337154           3029833842 2020-04-28 08:41:03+00:00   
76553   1276077305354432512           1010819870 2020-06-25 08:58:32+00:00   
49755   1161203634283192320             19811837 2019-08-13 09:11:35+00:00   
110701  1280034898762489856            176719569 2020-07-06 07:04:36+00:00   
51357   1188900278499774465            259733922 2019-10-28 19:28:09+00:00   
40377   1110528802248110081            340329223 2019-03-26 13:07:53+00:00   
31647   1062613180470476800            340329223 2018-11-14 07:48:19+00:00   
18968    889324366428811267           2317442264 2017-07-24 03:20:26+00:00   
112776  1278680221965856768  1079266247546753024 2020-07-02 13:26:24+00:00   
63558   1251505817226018816             77220394 2020-04-18 13:40:13+00:00   
65051   1263741813673918465  1159698800790953984 2020-05-22 08:01:42+00:00   
112950  1278639840335204352           1085664344 2020-07-02 10:43:55+00:00   
81367   1275091419678597120  1239515792011649024 2020-06-22 15:40:59+00:00   
1952    1225069953696632832   743798562593640448 2020-02-05 18:49:52+00:00   
53604   1179726384329048064            386165847 2019-10-03 11:54:22+00:00   
98611   1286397010556444672  1213095316657860608 2020-07-23 20:30:23+00:00   

                   tweet_id lang  retweet_count  reply_count  like_count  \
69503   1277525797243412480   de              0            0           0   
78607   1275541480271200258   de              0            0           1   
7263    1285302000780009481   de              0            0           0   
32865   1071003895101566977   de              0            0           0   
62348   1255054410343952384   de              0            0           1   
76553   1276077305354432512   de              0            0           1   
49755   1161203634283192320   de              0            0           1   
110701  1280034898762489856   de              1            0           1   
51357   1188900278499774465   de              0            0           0   
40377   1110528802248110081   de              0            0           0   
31647   1062613180470476800   de              0            0           0   
18968    889324366428811267   de              0            0           0   
112776  1278681431275974658   de              0            1           1   
63558   1251505817226018816   de              0            0           1   
65051   1263741813673918465   de              0            5           3   
112950  1278640539248799747   de              0            1           2   
81367   1275091419678597120   de              1            1           2   
1952    1225129428885409792   de              0            1           2   
53604   1179726384329048064   de              0            0           1   
98611   1286398273209733121   de              4            0           7   

        quote_count               source  tweet_type  referenced_tweet_id  \
69503             0      Blog2Social APP        None                 None   
78607             0     Twitter for iPad  replied_to  1275397576221765633   
7263              0   Twitter for iPhone  replied_to  1285301765789933570   
32865             0           Freie-Welt        None                 None   
62348             0      Twitter Web App  replied_to  1255054199756337154   
76553             0      Twitter Web App      quoted  1276074792597303298   
49755             0              Echobox        None                 None   
110701            0      Twitter Web App        None                 None   
51357             0               Google        None                 None   
40377 

<h2>Finalen Datensatz als CSV ausgeben.</h2>

In [42]:
df_de.to_csv('Dataset_Wirecard.csv', sep=';', index=False)

In [ ]:
df_validation_sentiment.head()

<h2>Erstellen der CSV zur Überprüfung der Sentiment Ergebnisse</h2>

In [57]:
df_de.sample(n=10)[["tweet_id", "text"]]

tweet_id                                               text
107505  1282624461716553728  Bei wirecard versagte noch VOR den Wirtschafts...
92212   1269968467828117506  Angesichts laufender Ermittlungen und der Durc...
5712    1273934114589155338   Rücktritt. Endlich. Ich gehe davon aus, dass ...
58198   1215973676904058880  Thomas Eichelmann wird neuer Aufsichtsratsvors...
89605   1273567326118596614   kein Problem. Wirecard sackt um 50% ab und au...
73670   1276259397061292032  Post wirecard. Was ist eure neue Anlagestrategie?
105557  1283828704750313477  Bundesfinanzminister Scholz hatte seit Februar...
77083   1276055514208755712  Bayer könnte wegen des Glyphosat-Vergleichs ei...
96547   1288015354686570496  Liebe dass Wirecard offenbar ein Ponzi Scheme ...
59205   1229322849891962885  🌏 Wirecard-Aktie schwach: Rechtsstreit mit Fin...

In [66]:
df_sentiment_validation = df_de.sample(n = 1500)[["tweet_id", "text"]]
batch = 14
counter = 0 
for batch in range(0,15):
    start = 100*batch
    end = 100*batch+99
    csv_name = 'de_validation_{count}.csv'.format(count=batch)
    df_sentiment_validation.iloc[start:end].to_csv(csv_name, sep=';', index=False)
    print("Start bei {start} und ende bei {ende}. Name der Datei: {csv}".format(start=start, ende=end, csv=csv_name))

Start bei 0 und ende bei 99. Name der Datei: de_validation_0.csv
Start bei 100 und ende bei 199. Name der Datei: de_validation_1.csv
Start bei 200 und ende bei 299. Name der Datei: de_validation_2.csv
Start bei 300 und ende bei 399. Name der Datei: de_validation_3.csv
Start bei 400 und ende bei 499. Name der Datei: de_validation_4.csv
Start bei 500 und ende bei 599. Name der Datei: de_validation_5.csv
Start bei 600 und ende bei 699. Name der Datei: de_validation_6.csv
Start bei 700 und ende bei 799. Name der Datei: de_validation_7.csv
Start bei 800 und ende bei 899. Name der Datei: de_validation_8.csv
Start bei 900 und ende bei 999. Name der Datei: de_validation_9.csv
Start bei 1000 und ende bei 1099. Name der Datei: de_validation_10.csv
Start bei 1100 und ende bei 1199. Name der Datei: de_validation_11.csv
Start bei 1200 und ende bei 1299. Name der Datei: de_validation_12.csv
Start bei 1300 und ende bei 1399. Name der Datei: de_validation_13.csv
Start bei 1400 und ende bei 1499. Name 

<h1>Weitere Überprüfungen der Daten</h1>
<p>Hier sind die Überprüfungen aufgeführt, die vorgenommen werden mussten um den Datensatz vorzubereiten. Das eigentliche Löschen der nicht benötigten Daten ist bereits weiter oben vorgenommen worden.</p>

<h2>Überprüfung der Wörter in den Tweets</h2>
<p>Ich prüfe wie oft die einzelnen Wörter in den jeweiligen Tweets vorkommen um ggf. nicht passende Wörter zu ermitteln.</p>
<p>Ladeb von deutschen Stop Wörtern um diese zu filter, da diese keinen wirklichen Mehrwert bieten. <br> Anschließend wird das Hanover Tagger Modell geladen um die Lemmatisierung durchzuführen um Wörter auf die Grundform zu reduzieren.</p>

In [ ]:
german_stop_words = stopwords.words('german')
tagger = ht.HanoverTagger('morphmodel_ger.pgz')

<p>Lemmatisierung um zu den jeweiligen Wörtern die Grundform zu erhalten. Dies wird mit den ganzen Tweets gemacht umd die Wörter im Kontext betrachten zu können (?). </p>

In [ ]:
tweet_list = df_manager_de["text"].tolist()
lemma_list = []

for tweet in tweet_list:
    lemma = [lemma for (word, lemma, pos) in tagger.tag_sent(tweet.split())]
    lemma_list.append(' '.join(lemma))

<p>Erstellen einer Liste mit allen Wörtern aus jeden einzelnen Tweet.</p>

In [ ]:
text_list = []
for each in lemma_list:
    for word in re.sub('\W+', ' ', each).lower().split():
        if word not in german_stop_words:
            text_list.append(word)

<p>Die Liste wird nun zu einer Pandas Serie um hier leichter Informationen zu erhalten. Im Anschluss wird die Serie zu einem Dataframe, bei dem die Spalten "Word" und "Count" heißen.</p>

In [ ]:
text_series = pd.Series(text_list)
df_word_count = text_series.value_counts().to_frame().reset_index().rename(columns={"index": "word", 0: "count"})

<p>Anzeigen von allen Wörtern, die öfter als 100 Mal in dem Datensatz vorkommen.</p>

In [ ]:
pd.set_option('display.max_rows', None)
df_word_count.loc[df_word_count["count"] >= 100]

<p>Überprüfung ob bei auffälligen Wörtern oder Kürzeln die Texte keinen Kontext zu Wirecard haben.</p>
<p>Auffällig Begriffe</p>
<lu>
    <li>FPÖ</li>
    <li>amp</li>
    <li>wdi</li>
</lu>

In [ ]:
for each in df_manager_de.loc[df_manager_de.text.str.lower().str.contains("fpö")]["text"]:
    print(each, '\n')

In [ ]:
for each in df_manager_de.loc[df_manager_de.text.str.lower().str.contains("amp")]["text"]:
    print(each, '\n')

In [ ]:
for each in df_manager_de.loc[df_manager_de.text.str.lower().str.contains("wdi")]["text"]:
    print(each, '\n')

<p>Der Begriff der in der Stichprobe keinen Bezug zu Wirecard hatte war <b>"FPÖ"</b>. Dementsprechend ist der Begriff oben in der ursprünlgichen Verarbeitung mit aufgenommen.</p>

<h2>Test prüfen nach Tweets ohne die Manager</h2>
<p>In diesem Abschnitt werden die die Tweets überprüft, in denen die Manager nicht vorkommen. Diese müssen Manuell überprüft werden und anschlißend auch im Kontext zugehöriger Tweets geprüft. <br>Wenn dabei herauskommen sollte, dass die Tweets nicht mit dem Thema zu tun haben, dann werden diese gelöscht. Dies wird bereits oben erledigt.</p>

<p>Ich prüfe auf verbleibende Tweets, in denen weder Markus Braun oder Jan Marsalek genannt werden. Ich prüfe dies auf verschiedene Schreibweisen um die Tweets zu ermitteln, in denen diese Namen nur in Links erwähnt wurden.</p>

In [ ]:
df_markus = df_manager_de.loc[df_manager_de.text.str.lower().str.contains("markus braun|markusbraun")].reset_index(drop=True)

In [ ]:
df_jan = df_manager_de.loc[df_manager_de.text.str.lower().str.contains("jan marsalek|janmarsalek")].reset_index(drop=True)

In [ ]:
df_markus.head()

In [ ]:
df_jan.head()

<p>Erhalten der Tweet_IDs und diese werden in einer Liste gespeichert.</p>

In [ ]:
list_markus_jan = []
for each in range(len(df_markus)):
    list_markus_jan.append(df_markus.iloc[each]["tweet_id"])
for each in range(len(df_jan)):
    list_markus_jan.append(df_jan.iloc[each]["tweet_id"])

In [ ]:
list_index = []
for each in list_markus_jan:
    list_index.append(df_manager_de.loc[df_manager_de["tweet_id"] == each].index[0])
    
df_man_with_mark_jan = df_manager_de.drop(list_index)
    #print(df_manager_de.loc[df_manager_de["tweet_id"] == each].index[0])
df_man_with_mark_jan = df_man_with_mark_jan.reset_index(drop=True)

In [ ]:
df_with_wirecard = df_man_with_mark_jan.loc[df_man_with_mark_jan.text.str.lower().str.contains("wirecard")]

In [ ]:
list_with_wirecard = []
for each in range(len(df_with_wirecard)):
    list_with_wirecard.append(df_with_wirecard.iloc[each]["tweet_id"])

In [ ]:
list_index = []
for each in list_with_wirecard:
    list_index.append(df_man_with_mark_jan.loc[df_man_with_mark_jan["tweet_id"] == each].index[0])
    #print(df_man_with_mark_jan.loc[df_man_with_mark_jan["tweet_id"] == each].index[0])
df_man_with_mark_jan = df_man_with_mark_jan.drop(list_index)
df_man_with_mark_jan = df_man_with_mark_jan.reset_index(drop=True)

<p>Prüfen von jedem Tweet, ob dieser einen Bezug zum Thema hat</p>

In [ ]:
for each in range(len(df_man_with_mark_jan)):
    print(df_man_with_mark_jan.iloc[each]["tweet_id"], ": ", df_man_with_mark_jan.iloc[each]["text"])

In [ ]:
print(len(df_man_with_mark_jan.drop_duplicates(subset=["text"])))

<p>Jeden Tweet, der als "auffällig" erkannt wurde mit zugehörigen Tweets manuell überprüfen</p>

In [ ]:
#Text Check up for Tweets without context.
tweet = 1130213579490234370
print("Original: ", df_man_with_mark_jan.loc[df_man_with_mark_jan["tweet_id"] == tweet]["text"].values[0], "\n")
for each in df_manager_de.loc[(
    df_manager_de["tweet_id"] == tweet) | (
    df_manager_de["conversation_id"] == df_manager_de.loc[df_manager_de["tweet_id"] == tweet]["conversation_id"].values[0]) | (
    df_manager_de["referenced_tweet_id"] == tweet) | (
    (df_manager_de["referenced_tweet_id"] == df_manager_de.loc[df_manager_de["tweet_id"] == tweet]["referenced_tweet_id"].values[0]) & (df_manager_de["referenced_tweet_id"] != "None")) | (
    df_manager_de["referenced_tweet_id"] == df_manager_de.loc[df_manager_de["tweet_id"] == tweet]["conversation_id"].values[0])]["text"].values:
    print(each, "\n")

<p>Ermittlung aller Tweets, die nichts mit dem Thema zu tun haben und die nach der manuellen Prüfung nicht in den Datensatz gehören. Alle Tweet_Ids werden in einer Liste gespeichert und im Anschluss gelöscht. Mit den bereits bekannten Tweet_IDs, wird dieser Prozess bereits weiter oben ausgeführt im Abschnitt "Tweets Löschen ohne Kontext"</p>

In [ ]:
#Text Check up for Tweet_id for removal
list_tweet_id = [701784983829143553, 909831998537158657, 909733713302441984, 909730731932368897, 1115628103487954944, 
                1115596390133051393, 1112624842493358080, 1131274228521742336, 1130213579490234370, 1207617572126044160, 
                1265176607905333248, 1273949458808680449, 1285646389771481088, 1285547240447246338, 1285515450584096768, 
                1285497151125151745, 1284925162836709378, 1279724802782216192]
list_tweet_id_remove = []
for tweet in list_tweet_id:
    for each in df_manager_de.loc[(
        df_manager_de["tweet_id"] == tweet) | (
        df_manager_de["conversation_id"] == df_manager_de.loc[df_manager_de["tweet_id"] == tweet]["conversation_id"].values[0]) | (
        df_manager_de["referenced_tweet_id"] == tweet) | (
        (df_manager_de["referenced_tweet_id"] == df_manager_de.loc[df_manager_de["tweet_id"] == tweet]["referenced_tweet_id"].values[0]) & (df_manager_de["referenced_tweet_id"] != "None")) | (
        df_manager_de["referenced_tweet_id"] == df_manager_de.loc[df_manager_de["tweet_id"] == tweet]["conversation_id"].values[0])]["tweet_id"].values:
        print(each, "\n")
        if each not in list_tweet_id_remove:
            list_tweet_id_remove.append(each)

<h1>TEST</h1>
<p>Hier werden unterschieldiche Funktionen getestet.</p>

In [ ]:
df_de.sample(n = 100)

In [ ]:
test_str_strInStr = "Das ist der Test ob Superkicker korrekt erkannt wird."

In [ ]:
if "kicker" in test_str_strInStr:
    print("Superkicker")

In [63]:
for each in range(0,15):
    print(each)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
